In [2]:
#pip install dash
#pip install dash-leaflet

In [6]:
from dash import Dash, dcc, html
from dash.dependencies import Input, Output, State
import plotly.express as px
import dash_leaflet as dl
import requests
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import mapping
import leafmap as leafmap
from shapely.geometry import shape
from dash_extensions.javascript import assign
import ipywidgets as widgets

In [7]:
# read_response checks if the response of the server is valid JSON

def read_response(t):
    try:
        data = t.json() #This will convert the response to a json object
        return data
    except requests.exceptions.JSONDecodeError:
        print("Risposta non valida JSON!")
        print("Contenuto della risposta:", t.text)
        data = None 

# Utility Function to Read API Responses

def read_response(response):
    try:
        return response.json()
    except requests.exceptions.JSONDecodeError:
        print("Invalid JSON response!")
        print("Response content:", response.text)
        return None


# Get List of Available Provinces

def get_provinces_list():
    response = requests.get("http://127.0.0.1:5000/api/provinces")
    return read_response(response) or []


# Get List of Available Municipalities

def get_municipalities_list():
    response = requests.get("http://127.0.0.1:5000/api/municipalities")
    return read_response(response) or []


# Get List of Available Pollutants

def get_pollutants_list():
    response = requests.get("http://127.0.0.1:5000/api/pollutants")
    return read_response(response) or []


# Call API to Get Data

def get_data(tipo, nome, pollutant):
    url = "http://127.0.0.1:5000/api/DV_7comune" if tipo == "Comune" else "http://127.0.0.1:5000/api/DV_7provincia"
    payload = {
        "var_comune": nome if tipo == "Comune" else None,
        "var_provincia": nome if tipo == "Provincia" else None,
        "var_pollutant": pollutant
    }
    payload = {k: v for k, v in payload.items() if v is not None}
    response = requests.post(url, json=payload)
    data = read_response(response)
    return pd.DataFrame(data) if data else pd.DataFrame()


# Dropdowns e Output UI (creazione degli elementi interattivi)

tipo_dropdown = widgets.Dropdown(
    options=["Comune", "Provincia"],
    description="Type:",
    value="Comune"
)

nome_dropdown = widgets.Dropdown(
    options=[],
    description="Name:"
)

pollutant_dropdown = widgets.Dropdown(
    options=get_pollutants_list(),
    description="pollutant:"
)

output = widgets.Output()